# A study on the timeseries forecasting method: ARIMA


## Getting the data

In [7]:
# initial imports
import pandas as pd
import numpy as np 

from TSErrors import FindErrors

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

import holidays
import calendar 

import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'holidays'

In [2]:
confirmed_global = pd.read_csv(r"./data/country_confirmed.csv")

In [3]:
def get_data(country, confirmed=confirmed_global):
    confirmed = confirmed.groupby("country").sum().T
    confirmed.index = pd.to_datetime(confirmed.index, infer_datetime_format=True)
    data = pd.DataFrame(
            index=confirmed.index, data=confirmed[country].values, columns=["Total"]
        )
    data = data[(data != 0).all(1)]

    data_diff = data.diff()

    # Removing the first value from data_diff 
    # It had no previous value and is a NaN after taking the difference
    data_diff = data_diff[1:]

    return data, data_diff

In [4]:
confirmed_dfs = get_data("India")
confirmed_daily = confirmed_dfs[1]

In [5]:
confirmed_daily.tail()

,Total
2022-03-04,5921.0
2022-03-05,5476.0
2022-03-06,4362.0
2022-03-07,3993.0
2022-03-08,4575.0


In [6]:
# plotting our daily cases
px.line(confirmed_daily, title="Daily Confirmed Cases in India")

## Model Identification

In [ ]:
# train test split
confirmed_daily.index.freq="D"
train = confirmed_daily[:int(len(confirmed_daily)*0.9)]
test = confirmed_daily[int(len(confirmed_daily)*0.9):]

There are 2 main things we need to check here while identifying our model:

1. If our time series data is stationary or not
2. Does our time series have a seasonal component or not

### Checking for stationary condition

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
acf = plot_acf(train["Total"], lags=20)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adfuller_test(ts):
    result = adfuller(ts)
    print(f'p-value: {result[1]}')
    print(f'Lags: {result[2]}')
    print(f'Number of obs: {result[3]}')


In [ ]:
adfuller_test(train["Total"])

As seen in the above tests:

1. The ACF plot decreases/declines slowly for different lags
2. The p-value returned by the Augmented Dicky-Fuller (adfuller) test is >0.05

Both of these observations are not consistent with the null hypothesis that the data is stationary. Therefore we **cannot use ARIMA(p,0,q)** or an ARMA model for our time series.

Our next step is to find the order of differencing for our ARIMA model to convert it into stationary data. i.e, to find d-value of the ARIMA(p,d,q) model.


In [ ]:
# trying with diff order = 1
train["total_diff"] = train["Total"].diff()

In [ ]:
train.head()

In [ ]:
train.dropna(inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train["total_diff"]))

In [ ]:
acf = plot_acf(train["total_diff"], lags=20)

In [ ]:
adfuller_test(train["total_diff"])

In order of differecning 1, we see that:

1. The ACF plot shows no sign of slow decline of autocorrelation over lags. It immediately drops.
2. The p value retuerned by the Dicky-Fuller test is extremely small, and well below the threshold of 0.05.

So with the differencing order of 1, we get a stationary time series data. Therefore we can use **ARIMA(p,1,q)** as our model.

In [ ]:
# finding p and q values of new staionary time series
from statsmodels.graphics.tsaplots import plot_pacf
pacf = plot_pacf(train["total_diff"], lags=20, method="ywm")

In [ ]:
acf = plot_acf(train["total_diff"], lags=20)

### Rules for deciding p and q

#### p value

1. The partial autocorrelation is significant for first p values/lags and then cuts off to zero
2. The ACF decreases exponentially

Point 2 is just a clarifier that the series needs to be stationary for us to select a p-value.
And from our pacf plot we identify that p = 1 (after p/lags=1, it cuts off to zero)

#### q value

1. The Autocorrelation is significant for first q values/lags and then cuts off to zero
2. The PACF decreases exponentially

Again point 2 is just a clarifier that the series needs to be stationary for us to select a q-value. And from our ACF plot we get q = 1

Therefore we have identified our model to be **ARIMA(1,1,1)**

### Checking for seasonality


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

ts_decompose = seasonal_decompose(train["Total"][:50], model="additive")
ts_plot = ts_decompose.plot()

In [ ]:
seasonal = ts_decompose.seasonal
fig = px.line(seasonal, title="Seasonality")
fig.show()

In [ ]:
adfuller_test(seasonal)

Therefore D = 0, as seasonal component is already stationary

In [ ]:
s_pacf = plot_pacf(seasonal, lags=20, method="ywm")

In [ ]:
s_acf = plot_acf(seasonal, lags=20)

In [ ]:
import statsmodels.api as sm
arima = sm.tsa.statespace.SARIMAX(train['Total'], order=(2,1,2), seasonal_order=(3,0,3,7),)

In [ ]:
arima_fit = arima.fit(maxiter=100000,method='nm')

In [ ]:
arima_fit.summary()

In [ ]:
pred = arima_fit.predict(start=train.index[-1], end=test.index[-1])[1:]

In [ ]:
mape = FindErrors(test.Total.values, pred.values).mape()

In [ ]:
mape

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train["Total"].values, name="Train"))
fig.add_trace(go.Scatter(x=test.index, y=test["Total"].values, name="Actual"))
fig.add_trace(go.Scatter(x=test.index, y=pred.values, name="Predicted"))